A notebook to calculate player elo. See: https://www.betfair.com.au/hub/tennis-elo-modelling for more details on the formula and implentation but hopefull the code is transparent enough that you can understand what is going on. I'm keeping the calculation separate from the main notebook because the computation of elo is somewhat time consuming.

The dataset is from the R package Deuce: https://github.com/skoval/deuce, so download the dataset from there first before running this notebook

In [1]:
import pandas as pd
import numpy as np

In [2]:
deuce_atp = pd.read_csv('Data/atp_matches_deuce.csv')

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
deuce_atp = pd.read_csv('Data/atp_matches_deuce.csv')
all_names = list(set(deuce_atp.loser_name.unique()) | set(deuce_atp.winner_name.unique()))

In [4]:
elo_dict = {player:[1500] for player in all_names}

i = 0

previous_winner_elos = []
previous_loser_elos = []

for index, row in deuce_atp.iterrows():
    
    winner = row.winner_name
    loser = row.loser_name
    
    winner_elos = elo_dict[winner]
    loser_elos = elo_dict[loser]
    
    winner_matches = len(winner_elos)
    loser_matches = len(loser_elos)
    
    winner_old_elo = winner_elos[-1]
    loser_old_elo = loser_elos[-1]
    
    previous_winner_elos.append(winner_old_elo)
    previous_loser_elos.append(loser_old_elo)
    
    pr_p1_win_elo = 1/(1+10**((loser_old_elo - winner_old_elo)/400))
    pr_p2_win_elo = 1/(1+10**((winner_old_elo - loser_old_elo)/400))
    
    winner_K = 250/((winner_matches+5)**0.4)
    loser_K = 250/((loser_matches+5)**0.4)
    
    winner_new_elo = winner_old_elo + winner_K*(1-pr_p1_win_elo)
    loser_new_elo = loser_old_elo + loser_K*(0-pr_p2_win_elo)
    
    winner_elos.append(winner_new_elo)
    loser_elos.append(loser_new_elo)
    
    elo_dict[winner] = winner_elos
    elo_dict[loser] = loser_elos

In [5]:
deuce_atp.loc[:,'winner_old_elo'] = previous_winner_elos
deuce_atp.loc[:,'loser_old_elo'] = previous_loser_elos

In [6]:
deuce_atp.to_csv('Data/atp_matches_elo.csv', index=False)